In [1]:
## standart libraries

import os,sys
import numpy as np

# xarray
import xarray as xr
import esmtools

In [2]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34305 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34305/status,
Dashboard: http://127.0.0.1:34305/status,Workers: 5
Total threads: 20,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45639,Workers: 5
Dashboard: http://127.0.0.1:34305/status,Total threads: 20
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:34727,Total threads: 4
Dashboard: http://127.0.0.1:34837/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:42223,


# load data

In [7]:
diro = "/gpfswork/rech/cli/uor98hu/MYDATA/SLdecomposition_WP1/Mano_from_Pb/"

In [3]:
g= 9.81      #(m/s2)
rho_0= 1026  #(kg/m-3) # axelle avait mis 1000
K= 10000   #(constante de conversion dbar to Pa)

In [4]:
# data for GMSL computation

In [5]:
# ------ mesh for constructing the weight for the computation of the global mean sea level
chunk_size2 = {"x":133,"y":320}
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(diri+'mesh_hgr.nc',chunks=chunk_size2).squeeze()
tmask = mesh_hgr.tmask

e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

nav_lon =  mesh_hgr.nav_lon
nav_lat =  mesh_hgr.nav_lat

# constructing the weight
bt = (e1t * e2t).load()# area of each cell
# getting the weights to be applied to every grid cell
# masking the land cells
oceanArea = (bt*tmask[0])  #tmask : 1 on ocean, 0 on land
totalOceanArea = oceanArea.sum()
Weight = (oceanArea/totalOceanArea).compute()


In [6]:
%%time
#-------- Read data
prefix = "eORCA025.L75-IMHOTEP"
varna = 'botpres'
fo="1m" 

# input directory on my work
chunk_size = {"x":133,"y":320}

for nexp in ["GAI"]:

    print(nexp)
    diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexp+"-S/"+fo+"/"
    data = xr.open_mfdataset(diridatref+'????/*_gridTsurf.nc', chunks = chunk_size)[varna].fillna(0).squeeze()

    # calculating the GMSL
    GMSL = (data*Weight).sum(('x','y')).compute()
    print('GMSL done')
    # correction of the GMSL
    dataCorrected = data - GMSL
    print('correction done')
    


#     #convert into dataset
#     sshEds = xr.Dataset(
#         data_vars=dict(botpres=(["time_counter", "y", "x"], dataCorrected.data),
#                       GMSL = (["time_counter"], GMSL.data) ),
#         coords=dict(time_counter=data.time_counter.values),
#         attrs=dict(
#             description=" sea level heigh corrected from the GMSL",
#             units="m",
#             interval_write = fo))

#     namo = prefix+'.'+nexp+'_y1980_2018.'+fo+'_sossheigCorrected.nc'
#     print(namo)
#     sshEds.to_netcdf(path='/gpfswork/rech/cli/uor98hu/MYDATA/SSH/'+nexp+'/'+namo, mode='w')


GAI
GMSL done
correction done
CPU times: user 2min 44s, sys: 12.6 s, total: 2min 57s
Wall time: 3min 51s


In [12]:
# convert to annual data
sshGAI =dataCorrected.groupby('time_counter.year').mean('time_counter').chunk(chunks={"x":500,"y":500, "year":20})

# retreive time mean
sshGAIprime = sshGAI - sshGAI.mean("year")

In [18]:
# convert to sea level
mano_prime = (sshGAIprime * K) / (rho_0 * g) # in meters

In [15]:
# years to read data from: 
y1='2005'
y2='2018'

In [20]:
%%time
GAI_4trend = ( mano_prime.sel(year=slice(y1,y2)) ).compute()

In [21]:
GAI_4trend

<xarray.DataArray (year: 14, y: 1207, x: 1442)>
array([[[-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028],
        [-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028],
        [-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028],
        ...,
        [-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028],
        [-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028],
        [-0.00136028, -0.00136028, -0.00136028, ..., -0.00136028,
         -0.00136028, -0.00136028]],

       [[-0.00401046, -0.00401046, -0.00401046, ..., -0.00401046,
         -0.00401046, -0.00401046],
        [-0.00401046, -0.00401046, -0.00401046, ..., -0.00401046,
         -0.00401046, -0.00401046],
        [-0.00401046, -0.00401046, -0.00401046, ..., -0.00401046,
         -0.00401046, -0.00401046],
...
        [-0.01593321, -0.01593321, -0.01593321, ..., -0.01593321,
         -0.01593321, -0.01593321],
        [-0.01593321, -0.01593321, -0.01593321, ..., -0.01593321,
         -0.01593321, -0.01593321],
        [-0.01593321, -0.01593321, -0.01593321, ..., -0.01593321,
         -0.01593321, -0.01593321]],

       [[-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855],
        [-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855],
        [-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855],
        ...,
        [-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855],
        [-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855],
        [-0.01227855, -0.01227855, -0.01227855, ..., -0.01227855,
         -0.01227855, -0.01227855]]])
Coordinates:
    nav_lat  (y, x) float32 -84.2 -84.2 -84.2 -84.2 ... 50.07 50.02 50.0 50.0
    nav_lon  (y, x) float32 72.75 73.0 73.25 73.5 ... 73.01 73.01 73.0 73.0
    nav_lev  float32 0.5058
  * year     (year) int64 2005 2006 2007 2008 2009 ... 2014 2015 2016 2017 2018
Dimensions without coordinates: y, x

In [22]:
%%time
lin_GAI = esmtools.stats.linregress((GAI_4trend*1000), dim="year") # pour avoir en mm/yr

/gpfslocalsup/pub/anaconda-py3/2020.11/envs/climate_science/lib/python3.9/site-packages/esmtools/stats.py:394: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  results = xr.apply_ufunc(


CPU times: user 9min 45s, sys: 1min 13s, total: 10min 58s
Wall time: 9min 34s


In [24]:
# convert into dataset and save : GAI

ds2save = xr.Dataset(data_vars=dict(linregress=(["y", "x","parameter"], lin_GAI.data)),
                            coords=dict(
                                parameter = (["parameter"],['slope', 'intercept', 'rvalue', 'pvalue', 'stderr'])),
                             attrs=dict(description="bottom pressure - Global mean - time mean, then linear trend over 2005-2018 (mm/yr) calculated over yearly data, saved the 6 fev 2024"))

ds2save.to_netcdf(diro+"eORCA025.L75-IMHOTEP._y2005_2018.1y_trend_mano_GAI_linregress.nc")

# vérif

cf notebbok Plot